# 3. Using Included ProgPy Models

ProgPy is distributed with a few pre-constructed models that can be used in simulation or prognostics. These models for batteries, pumps, valves, among others, are included in the `progpy.models` package.

In this notebook, we will be exploring a generalized overview of each included model. For more in-depth descriptions of the included models, please refer to the [Included Models](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html) documentation.

## Table of Contents
* [Battery Models](#Battery-Models)
    * [Battery Circuit](#BatteryCircuit)
    * [BatteryElectroChemEOD](#BatteryElectroChemEOD)
    * [BatteryElectroChemEOL](#BatteryElectroChemEOL)
    * [Combined BatteryElectroChem (BatteryElectroChemEODEOL)](#Combined-BatteryElectroChem-(BatteryElectroChemEODEOL))
* [Centrifugal Pump Model](#Centrifugal-Pump-Model)
* [Electric Powertrain Models](#Electric-Powertrain-Models)
* [Pneumatic Valve Model](#Pneumatic-Valve-Model)
* [Aircraft Flight Model](#Aircraft-Flight-Model)

## Battery Models

In the following battery models, the default model parameters included are for Li-ion batteries, specifically 18650-type cells. Experimental discharge curves for these cells can be downloaded from the Prognostics Center of Excellence [Data Repository](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository).

### BatteryCircuit

In this first example, we will demonstrate how to set up, configure, and use the `BatteryCiruit` model. The `BatteryCircuit` model is a vectorized prognostics model for a battery, represented by an equivalent circuit model as described in [[Daigle Sankararaman 2013]](https://papers.phmsociety.org/index.php/phmconf/article/view/2253).

We will start by importing the model and initializing a battery instance with default settings.

In [ ]:
from progpy.models import BatteryCircuit
import matplotlib.pyplot as plt

batt = BatteryCircuit()

Information is passed to and from the model using containers that function like dictionaries. The keys of the containers are specific to the model. Let's look at the inputs (loading) and outputs (measurements) for the `BatteryCircuit` model.

In [ ]:
print('inputs:', batt.inputs)
print('outputs:', batt.outputs)

If we refer to the [documentation](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html), we can see that the input `i` refers to the current draw on the battery. The outputs `t` refers to the temperature in units Kelvin and `v` refers to voltage.

We can also print out what events we're predicting and the internal states the model uses to represent the system.

In [ ]:
print('event(s): ', batt.events)
print('states: ', batt.states)

We can see that this particular model only predicts one event, called `EOD` (End of Discharge). The states listed include `tb`, the battery temperature in K; `qb`, the charge stored in Capacitor Cb of the equivalent circuit model; `qcp`, the charge stored in Capacitor Ccp of the equivalent circuit model; and `qcs`, the charge stored in Capacitor Ccs of the equivalent circuit model.

Let's now look at the model's configuration parameters, which describe the specific system (in this case, the battery) that the model is simulating.

In [ ]:
from pprint import pprint

print('Model configuration:')
pprint(batt.parameters)

Let's now use the model to do a simulation. To do this, we will first need to set a configuration and define a future load. For more details on future loading, refer to the related section in __[01 Simulation](01_Simulation.ipynb#Future-Loading)__.

In [ ]:
config = {
    'save_freq': 100,
    'dt': 2,
    't0': 700
}

def future_loading(t, x=None):
    if (t < 600):
        i = 2
    elif (t < 900):
        i = 1
    elif (t < 1800):
        i = 4
    elif (t < 3000):
        i = 2     
    else:
        i = 3
    return batt.InputContainer({'i': i})

Let's run the simulation and plot the inputs and outputs. We can do this using the built-in [plot method](https://nasa.github.io/progpy/api_ref/progpy/SimResult.html#progpy.sim_result.SimResult.plot) based on matplotlib or with other imported plotting libraries.

In the inputs plot, we can see the current drawn change based on the logic we defined in the future loading function.

In [ ]:
simulated_results = batt.simulate_to_threshold(future_loading, **config)

fig = simulated_results.inputs.plot(xlabel='time (s)', ylabel='current draw (amps)')

In the outputs plot, we can observe how different input current draws affect the temperature and voltage curves. Generally, the graphs indicate that drawing a higher current leads to higher temperatures and lower voltage.

In [ ]:
fig = simulated_results.outputs.plot(keys=['t'], xlabel='time (s)', ylabel='temperature (K)', figsize=(10,4))
fig2 = simulated_results.outputs.plot(keys=['v'], xlabel='time (s)', ylabel='voltage (V)', figsize=(10,4))

### BatteryElectroChemEOD

`BatteryElectroChemEOD` is a vectorized prognostics model for a battery, represented by an electrochemical equations as described in [[Daigle 2013]](https://papers.phmsociety.org/index.php/phmconf/article/view/2252). This model predicts the end of discharge event. Let's start by examining the model inputs, outputs, event(s), and states. We can refer to the [documentation](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html) for more details.

In [ ]:
from progpy.models import BatteryElectroChemEOD

batt = BatteryElectroChemEOD()

print('inputs:', batt.inputs)
print('outputs:', batt.outputs)
print('event(s): ', batt.events)
print('states:', batt.states)

Let's now run a simulation until `EOD`, or end of discharge. We wil use the same future loading function as the previous example and specify the threshold event as `EOD`.

In the input plot, we can see the current draw change based on the future loading function we defined.

In [ ]:
simulated_results = batt.simulate_to_threshold(future_loading, events=['EOD'])

fig = simulated_results.inputs.plot(xlabel='time (s)', ylabel='current draw (amps)')

In the output plots, we can see changes in voltage and temperature. We can also print parameters like `VEOD`, or the end of discharge voltage threshold. This value is the voltage at which a battery is considered fully discharged.

In [ ]:
fig = simulated_results.outputs.plot(keys=['v'], xlabel='time (s)', ylabel='voltage (V)', figsize=(10,4))
print("End of discharge voltage threshold:", batt.parameters['VEOD'])

fig2 = simulated_results.outputs.plot(keys=['t'], xlabel='time (s)', ylabel='temperature (°C)', figsize=(10,4))

In the event state plot, we can see `EOD` decline until it reaches 0, or when the end of discharge event has occurred. This event occurence is when the simulation reached threshold and ended.

In [ ]:
fig = simulated_results.event_states.plot(xlabel='time (s)', ylabel='event state', labels={'EOD'})

### BatteryElectroChemEOL

`BatteryElectroChemEOL` is a vectorized prognostics model for battery degradation, represented by an electrochemical model as described in [[Daigle 2016]](https://arc.aiaa.org/doi/pdf/10.2514/6.2016-2132). Let's go ahead and import the model, initialize a battery instance, and take a closer look at the details. We can also refer to the [documentation](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html). Note that the model has no outputs. 

In [ ]:
from progpy.models import BatteryElectroChemEOL

batt = BatteryElectroChemEOL()

print('inputs:', batt.inputs)
print('outputs:', batt.outputs)
print('event(s): ', batt.events)
print('states:', batt.states)

Let's now run a simulation to predict when we will reach insufficient battery capacity. We will use the same future loading function as the previous examples and specify the threhold event as `InsufficientCapacity`.

In the input plot, we can once again see the current draw change based on the future loading function we defined.

In [ ]:
simulated_results = batt.simulate_to_threshold(future_loading, events=['InsufficientCapacity'])

fig = simulated_results.inputs.plot(xlabel='time (s)', ylabel='current draw (amps)')

In the event state plot, we can see `InsufficientCapacity` linearly decrease until it reaches 0, or when the event has occurred.

In [ ]:
fig = simulated_results.event_states.plot(xlabel='time (s)', ylabel='event state', labels={'InsufficientCapacity'})

### Combined BatteryElectroChem (BatteryElectroChemEODEOL)

`BatteryElectroChemEODEOL` is a prognostics model for battery degradation and discharge, represented by an electrochemical model as described in [[Daigle 2013]](https://papers.phmsociety.org/index.php/phmconf/article/view/2252) and [[Daigle 2016]](https://arc.aiaa.org/doi/pdf/10.2514/6.2016-2132). This model combines both the `BatteryElectroChemEOL` and `BatteryElectroChemEOD` models.

We will start by importing the model, initializing a battery instance, and examining the model details. We can once again refer to the [documentation](https://nasa.github.io/progpy/api_ref/progpy/IncludedModels.html) for more details.

In [ ]:
from progpy.models import BatteryElectroChem
import matplotlib.pyplot as plt

batt = BatteryElectroChem()

print('inputs:', batt.inputs)
print('outputs:', batt.outputs)
print('event(s): ', batt.events)
print('states:', batt.states)

In this example, we will simulating a battery until `EOL` (End of Life). As battery capacity decreases with use, `EOL` is when the battery capacity falls below some acceptable threshold (i.e., what we define as useful capacity).

We will now set the configuration and define a future loading function. As we want to simulate until `EOL`, we will set the configuration event to `InsufficientCapacity`. The future loading function is designed to charge the battery until `EOD` is 0.95 and then discharge until `EOD` is 0.05. Note that states represent the progress towards the event occurring. An event state of 0 indicates the event has occurred and 1 indicates no progress towards the event.

In [ ]:
config = {
    'save_freq': 1000,
    'dt': 2,
    'events': 'InsufficientCapacity',
}

def future_loading(t, x=None):
    load = 1
    
    if x is not None:
        event_state = batt.event_state(x)
        if event_state["EOD"] > 0.95:
            load = 1 # Discharge
        elif event_state["EOD"] < 0.05:
            load = -1 # Charge
            
    return batt.InputContainer({'i': load})

We will now simulate to the threshold and print the results.

In [ ]:
simulated_results = batt.simulate_to_threshold(future_loading, **config)

We will now plot the inputs, outputs, and event states. In the input plot, we can see the current drawn fluctuate between -1 and 1 based on the current load we defined in the future loading function.

In [ ]:
fig = simulated_results.inputs.plot(xlabel='time (s)', ylabel='current drawn (amps)')

In the output plots, we can see changes in the voltage and temperature.

In [ ]:
fig = simulated_results.outputs.plot(keys=['v'], xlabel='time (s)', ylabel='voltage (V)', figsize=(10,4))
fig2 = simulated_results.outputs.plot(keys=['t'], xlabel='time (s)', ylabel='temperature (°C)', figsize=(10,4))

In the event states plot, we can see `EOD` incrementally spiking and `InsufficientCapacity` linearly declining until it reaches 0, or when the event has occurred.

In [ ]:
fig = simulated_results.event_states.plot(xlabel='time (s)', ylabel='event states', labels={'EOD', 'InsufficientCapacity'})

## Centrifugal Pump Model

## Electric Powertrain Models

## Pneumatic Valve Model

## Aircraft Flight Model